# Build Functions Script
Name is WIP

## Goals:
- Build a deployable script that can reference and update an online list of functions, as well as build a .py file with these functions

## Stakeholders:
- Mostly me, but it could prove to be super useful to others too

## Deliverables:
- function viewer + editor:
    - create a dictionary object containing details about a function
    - create a list of dictionary objects, where each item is a function
    - edit and remove functions
    - show information about a function (to include dependencies and syntax)
- library viewer + editor:
    - load a list of Python libraries and modules
    - show information about these libraries and modules
    - add and edit these libraries and modules
- module builder:
    - get from the user the name of the file to create
    - check if the file given matches any tags associated with the functions list
    - suggest functions based on their tags
- function assembler:
    - get functions to include from the user
    - assemble a list of functions 
- library assembler:
    - get dependencies for a given function
    - assemble a list of dependencies for a given function (probably using a tuple)
- grand function:
    - runs the above in some fashion

In [2]:
# # functions example:
# functions = [
# 	{
# 		'function_name':'function_call_name',
# 		'modules':['module for function']
# 		'description':'description of function actions',
# 		'dependencies':{
# 			'name':'type'
# 		},
# 		'syntax':'code to create the function''
# 	}
# ]

# # libraries example
# libraries = [
# 	{
# 		'name':'library/module name',
# 		'syntax':'library/module syntax'
# 	}
# ]

In [1]:
# establish imports
import base
from library import manage_library
from function import manage_functions

In [3]:
manage_library()

What would you like to do?



        1) Add a new entry
        2) Edit an existing one
        3) View a list of current libraries
        4) View a specific library
        5) Exit
         5


In [5]:
manage_functions()

What would you like to do?



        1) Add a new entry
        2) Edit an existing one
        3) Remove a specific function
        4) View a list of current functions
        5) View a specific function
        6) Exit
         6


### Function 0.1: Data Saver
**Idea**: This function will be the crux of operations. Most operations would involve this function, as they require the data stored within.

**Final Product**: A function that by default returns None. If no arguments are given, it does nothing. If a DataFrame is provided, then one of its other arguments, 'library' or 'function' must be True. If 'library' or 'function' is True, then it will perform the operations of loading the specific data and returning it.

Parameters:
- load_library=False
    - if True then checks if library csv exists. If library csv exists, then load and return it. If it does not exist, then make it and return starter DataFrame with 'name', 'syntax', and 'desc' columns.
- load_function=False
    - if True, then checks if functions json exists. If functions json exists, then load and return it. If it does not exist, then make it and return starter DataFrame with the 'name','desc','syntax', and 'dependencies' columns.
- saved_data=None
    - if not None, then checks number of columns. If length of columns matches library list, then overwrite libraries.csv. If length of columns matches functions list, then overwrite functions.json. Else print that length matches neither.

In [9]:
def data_saver(load_library=False,load_function=False,saved_data=None):
    # check if load_library is true
    if load_library == True:
        # check if libraries.csv exists
        if os.path.exists('libraries.csv') == True:
            libraries = pd.read_csv('libraries.csv',index_col=0)
            return libraries
        else:
            libraries = pd.DataFrame(columns=['name','syntax','desc'])
            libraries.to_csv('libraries.csv')
            return libraries
    
    # check if load_function is true
    elif load_function == True:
        if os.path.exists('functions.json') == True:
            functions = pd.read_json('functions.json')
            return functions
        else:
            functions = pd.DataFrame(columns=['name','desc','tags','dependencies','syntax'])
            functions.to_json('functions.json')
            return functions
    
    # check if saved_data is not None
    elif type(saved_data) == type(pd.DataFrame()):
        if len(saved_data.columns) == 3:
            saved_data.to_csv('libraries.csv')
        elif len(saved_data.columns) == 5:
            saved_data.to_json('functions.json')
        else:
            print('Saved data length matches none')
    
    else:
        print('No options provided. Exiting.')
    
    return None

In [11]:
data_saver(load_library=True)

,name,syntax,desc
0,pandas,import pandas as pd,data manipulation library


### Function 0.2: Format text with embedded newlines
**Problem:** So at the function editor, I encountered the problem that copying and pasting a function results in a mess of formatting. Newlines aren't kept, so everything is globbed together in an ugly mess. This won't work for when the function begins to build the modules, so I need to correct this issue now so that everything will format correctly.

**Solution:** The following function relies on the *re* module. It finds any 3 or more consecutive spaces and adds a newline character to the string before the spaces. This way, any text entered will, when printed, be written with the appropriate indentations and should work as intended.

In [48]:
def add_newline_for_spaces(text):
    # Replace two or more spaces with a newline followed by the spaces
    modified_text = re.sub(r'(\S)(\s{3,})', r'\1\n\2', text)
    return modified_text

### Function 1: Library Viewer and Editor
**Idea**: We all have to start somewhere, and why not with the list of dependencies? I can start to see how things are building and perfect this piece, and in turn this should help with the functions builder, since that will be on a larger scale.

**Final Product**: A function capable of loading a list of libraries. These should not be duplicating. The function should build a dictionary for each module or library. The dictionary should include a *name*, *brief description*, and *syntax* about the library/module. It should also allow for editing of these items.

In [14]:
library_list

,name,syntax,desc


In [16]:
# # add to the list
# library_list.loc[-1] = [item_type,item_name,item_syntax,item_desc]

# # reset the index
# library_list = library_list.reset_index(drop=True)

In [18]:
library_list

,name,syntax,desc


In [141]:
def manage_library():
    df = data_saver(load_library=True)
    
    while True:
        time.sleep(0.05)
        clear_output()
        # Ask the user what they want to do
        print('What would you like to do?')
        action = input("""
        1) Add a new entry
        2) Edit an existing one
        3) View a list of current libraries
        4) View a specific library
        5) Exit
        """)

        if action == '1':
            # Add a new entry
            name = input("Enter the name of the module: ")
            syntax = input("Enter the syntax: ")
            desc = input("Enter the description: ")
            new_data = {'name': name, 'syntax': syntax, 'desc': desc}
            df.loc[-1] = new_data

        elif action == '2':
            # Edit an existing entry
            name_to_edit = input("Enter the name of the module to edit: ")

            if name_to_edit in df['name'].values:
                new_name = input("Enter the new name (or press enter to keep it the same): ")
                new_syntax = input("Enter the new syntax (or press enter to keep it the same): ")
                new_desc = input("Enter the new description (or press enter to keep it the same): ")

                if new_name: df.loc[df['name'] == name_to_edit, 'name'] = new_name
                if new_syntax: df.loc[df['name'] == name_to_edit, 'syntax'] = new_syntax
                if new_desc: df.loc[df['name'] == name_to_edit, 'desc'] = new_desc
            else:
                print(f"No module named {name_to_edit} found.")
        
        elif action == '3':
            # view current libraries
            for i in df.index:
                print(f'{df.iloc[i]["name"]}: {df.iloc[i].desc}')
            input('Press enter to continue')
        
        elif action == '4':
            # view specific library
            df = data_saver(load_library=True)

            for i in df.index:
                print(f'{df.iloc[i]["name"]}')

            library_to_view = input('Which would you like to see? ')
            
            # pull the library index
            item_index = df[df["name"] == library_to_view].index

            for col in df.columns:
                print(f'{col}: {df[df["name"] == library_to_view][col][item_index[0]]}')
            
            input('Press enter to continue')
        
        elif action == '5':
            # exit the program/loop
            break

        else:
            print("Invalid action. Please choose a valid option.")
            time.sleep(1.25)
        
        df = df.drop_duplicates(subset='name', keep='first').reset_index(drop=True)
    
    data_saver(saved_data=df)

    return None

In [137]:
manage_library()

What would you like to do?



        1) Add a new entry
        2) Edit an existing one
        3) View a list of current libraries
        4) View a specific library
        5) Exit
         5


In [245]:
libraries = data_saver(load_library=True)

In [299]:
libraries[libraries.name == 'pandas'].name[0]

'pandas'

In [111]:
library_to_view = 'time'

In [129]:
item_index = df[df["name"] == library_to_view].index
item_index[0]

1

In [ ]:
 {df[df["name"] == library_to_view][col][0]}

In [139]:
df = data_saver(load_library=True)

for i in df.index:
    print(f'{df.iloc[i]["name"]}')

library_to_view = input('Which would you like to see? ')

item_index = df[df["name"] == library_to_view].index

for col in df.columns:
    print(f'{col}: {df[df["name"] == library_to_view][col][item_index[0]]}')

pandas
time
numpy
tqdm
matplotlib
seaborn


Which would you like to see?  tqdm


name: tqdm
syntax: from tqdm import tqdm
desc: progress bars for iterative operations


### Function 2: Function viewer and Editor
**Idea**: This function will mostly expand on the library viewer and editor capabilities. It will operate fairly similar, just with some added components.

**Final Product**: A function capable of loading a list of functions. These should not be duplicating. The function should allow for adding a function, editing an existing function, and removing an existing function. It should also display information about a function (to include a description, the dependencies, the tags, and the syntax).

In [162]:
def manage_functions():
    df = data_saver(load_function=True)
    
    while True:
        time.sleep(0.05)
        clear_output()
        # Ask the user what they want to do
        print('What would you like to do?')
        action = input("""
        1) Add a new entry
        2) Edit an existing one
        3) Remove a specific function
        4) View a list of current functions
        5) View a specific function
        6) Exit
        """)
        
        if action == '1':
            # add a new entry
            # print('Option 1 not yet available')
            # time.sleep(1.25)
            
            # Add a new entry
            name = input("Enter the name of the module: ")
            syntax = add_newline_for_spaces(input("Enter the syntax: "))
            desc = input("Enter the description: ")
            dependencies_in = input("Enter dependency type followed by name of dependency (e.g. 'library:pandas')\n \
      If multiple dependencies, separate each item with a comma.")
            # tags = input("Enter modules that this function may be associated with: ")
            
            # Format the dependencies appropriately
            dependencies = {}
            
            # Split input_string into pairs and strip spaces
            pairs = [pair.strip() for pair in dependencies_in.split(',')]

            for pair in pairs:
                key, value = map(str.strip, pair.split(':'))
                if key in dependencies:
                    dependencies[key].append(value)
                else:
                    dependencies[key] = [value]
            
            new_data = {'name': name,
                        'desc': desc,
                        # 'tags': tags,
                        'dependencies': dependencies,
                        'syntax': syntax
                       }
            df.loc[-1] = new_data
        
        elif action == '2':
            # Edit an existing one
            print('Option 2 not yet available')
            time.sleep(1.25)
        
        elif action == '3':
            # Remove a specific function
            print('Option 3 not yet available')
            time.sleep(1.25)
        
        elif action == '4':
            # View a list of current functions
            # print('Option 4 not yet available')
            # time.sleep(1.25)
            
            for i in df.index:
                print(f'{df.iloc[i]["name"]}: {df.iloc[i].desc}')
            input('Press enter to continue')
        
        elif action == '5':
            # View a specific function
            # print('Option 5 not yet available')
            # time.sleep(1.25)
            
            for i in df.index:
                print(f'{df.iloc[i]["name"]}')

            function_to_view = input('Which would you like to see? ')

            item_index = df[df["name"] == function_to_view].index

            for col in df.columns:
                print(f'{col}: {df[df["name"] == function_to_view][col][item_index[0]]}')
            
            input('Press enter to continue')
        
        elif action == '6':
            # Exit
            break
        
        else:
            print("Invalid action. Please choose a valid option.")
            time.sleep(1.25)
    
        df = df.drop_duplicates(subset='name', keep='first').reset_index(drop=True)
        
    data_saver(saved_data=df)
    
    return None

In [158]:
manage_functions()

What would you like to do?



        1) Add a new entry
        2) Edit an existing one
        3) Remove a specific function
        4) View a list of current functions
        5) View a specific function
        6) Exit
         6
